In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Question_Pairs_BiLSTM

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_train = pd.read_csv('data/train_dataset.csv')
data_val = pd.read_csv('data/val_dataset.csv')

### Bert WordPiece Tokenizer

In [ ]:
!pip install -q transformers

In [ ]:
from tokenizers import BertWordPieceTokenizer
import os

In [ ]:
f = open('data/questions.raw', 'w', encoding='utf-8')
for i in data_train.question1.to_list() + data_train.question2.to_list() + data_val.question1.to_list() + data_val.question2.to_list():
    f.write(i)
    f.write('\n')
f.close()

In [ ]:
tokenizer = BertWordPieceTokenizer(lowercase=True)

In [ ]:
tokenizer.train('data/questions.raw')

In [ ]:
tokenizer.save_model('data/')

In [ ]:
# test
print('tokens: {}'.format(tokenizer.encode("I'am Crazy Man!!!").tokens))
print('ids: {}'.format(tokenizer.encode("I'am Crazy Man!!!").ids))

### Data Preprocessing

In [ ]:
from torch.utils import data
import torch
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class CustomDataset(data.Dataset):
    def __init__(self, docs_1, docs_2, labels, tokenizer):
        self.docs_1 = docs_1
        self.docs_2 = docs_2
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.docs_1)

    def __getitem__(self, item):
        docs_1 = str(self.docs_1[item])
        docs_2 = str(self.docs_2[item])
        labels = int(self.labels[item])
        encoded_docs_1 = self.tokenizer.encode(docs_1)
        encoded_docs_2 = self.tokenizer.encode(docs_2)
        return dict(
            input_ids_1 = torch.tensor(encoded_docs_1.ids),
            input_ids_2 = torch.tensor(encoded_docs_2.ids),
            labels = labels
            )

In [ ]:
train_data = CustomDataset(
    data_train.question1.to_numpy(),
    data_train.question2.to_numpy(), 
    data_train.is_duplicate.to_numpy(),
    tokenizer
    )
validation_data = CustomDataset(
    data_val.question1.to_numpy(),
    data_val.question2.to_numpy(), 
    data_val.is_duplicate.to_numpy(),
    tokenizer
    )

In [ ]:
def padding(batch):
    q1 = [q1['input_ids_1'] for q1 in batch]
    q2 = [q2['input_ids_2'] for q2 in batch]
    t = [t['labels'] for t in batch]
    len_q1 = [len(q1['input_ids_1']) for q1 in batch]
    len_q2 = [len(q2['input_ids_2']) for q2 in batch]
    q1_pad = pad_sequence(q1, batch_first=True, padding_value=0)
    q2_pad = pad_sequence(q2, batch_first=True, padding_value=0)
    return q1_pad, q2_pad, torch.tensor(t), len_q1, len_q2

In [ ]:
train_loader = data.DataLoader(train_data, batch_size = 64, shuffle=True, collate_fn=padding)
validation_loader = data.DataLoader(validation_data, batch_size = 128, collate_fn=padding)

In [ ]:
for q1_pad, q2_pad, t, len_q1, len_q2 in train_loader:
    print('questions_1: {}, shape: {}'.format(q1_pad, q1_pad.shape))
    print('')
    print('len_questions_1: {}'.format(len_q1))
    print('')
    print('questions_2: {}, shape: {}'.format(q2_pad, q2_pad.shape))
    print('')
    print('len_questions_2: {}'.format(len_q2))
    print('')
    print('lables: {}, shape: {}'.format(t, t.shape))
    break

In [ ]:
for q1_pad, q2_pad, t, len_q1, len_q2 in validation_loader:
    print('questions_1: {}, shape: {}'.format(q1_pad, q1_pad.shape))
    print('')
    print('len_questions_1: {}'.format(len_q1))
    print('')
    print('questions_2: {}, shape: {}'.format(q2_pad, q2_pad.shape))
    print('')
    print('len_questions_2: {}'.format(len_q2))
    print('')
    print('lables: {}, shape: {}'.format(t, t.shape))
    break

### Bidirectional LSTM

In [ ]:
import torch.nn as nn
device = "cuda:0" if torch.cuda.is_available() else "cpu"
!nvidia-smi

In [ ]:
class BiLSTM(nn.Module):
  def __init__(self, n_vocabs, embed_dims, n_lstm_units, n_lstm_layers, n_output_classes):
    super(BiLSTM, self).__init__()
    self.v = n_vocabs
    self.e = embed_dims
    self.u = n_lstm_units
    self.l = n_lstm_layers
    self.o = n_output_classes
    self.padd_idx = tokenizer.get_vocab()['[PAD]']
    self.embed = nn.Embedding(
        self.v,
        self.e,
        self.padd_idx
        )
    self.bilstm = nn.LSTM(
        self.e,
        self.u,
        self.l,
        batch_first = True,
        bidirectional = True,
        dropout = 0.5
        )
    self.linear = nn.Linear(
        self.u * 4,
        self.o
        )  
  
  def forward(self, X):
    # initial_hidden
    h0 = torch.zeros(self.l * 2, X[0].size(0), self.u).to(device)
    c0 = torch.zeros(self.l * 2, X[0].size(0), self.u).to(device)
    
    # embedding
    out1 = self.embed(X[0].to(device))
    out2 = self.embed(X[1].to(device))

    # pack_padded_sequence
    out1 = nn.utils.rnn.pack_padded_sequence(out1, torch.tensor(X[3]).to(device), batch_first=True, enforce_sorted=False)
    out2 = nn.utils.rnn.pack_padded_sequence(out2, torch.tensor(X[4]).to(device), batch_first=True, enforce_sorted=False)
    
    # NxTxh, lxNxh
    out1, _ = self.bilstm(out1, (h0, c0))
    out2, _ = self.bilstm(out2, (h0, c0))
    
    # pad_packed_sequence
    out1, _ = nn.utils.rnn.pad_packed_sequence(out1, batch_first=True)
    out2, _ = nn.utils.rnn.pad_packed_sequence(out2, batch_first=True)

    # take only the final time step
    out1 = out1[range(out1.shape[0]), torch.tensor(X[3]).to(device) - 1, :]
    out2 = out2[range(out2.shape[0]), torch.tensor(X[4]).to(device) - 1, :]
    
    # concatenate out1&2
    out = torch.cat((out1, out2), 1)
    
    # linear layer
    out = self.linear(out)

    iout = torch.max(out, 1)[1]
    return iout, out

In [ ]:
train_labels = torch.cat([i[2] for i in train_loader])
negative = (train_labels == 0.).sum(dim=0).item()
positive = (train_labels == 1.).sum(dim=0).item()

In [ ]:
print(f'number data in training set {len(train_data)}')
print(f'number of negative data in training set {negative}')
print(f'number of positive data in training set {positive}')

class_weight :

n = 203998 * (119431 / (119431 + 203998)) = 75329.3153613312

p = 119431 * (203998 / (119431 + 203998)) = 75329.3153613312

In [ ]:
torch.manual_seed(42)
model = BiLSTM(tokenizer.get_vocab_size(), 512, 512, 2, 2).to(device)
class_weight = torch.tensor([positive/(positive+negative), negative/(positive+negative)]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weight, reduction='mean').to(device)
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr = 1e-3
    )

In [ ]:
print(model)

In [ ]:
import sys
from tqdm import tqdm
from datetime import datetime

num_epochs = 5
losses = []
accuracies  = []
val_losses = []
val_accuracies = []
val_loss_min = np.inf

for epoch in range(num_epochs):
    print('------------------------------------------------------------------------------------------')
    print('epoch: {}/{}:'.format(epoch + 1, num_epochs))   
    print('------------------------------------------------------------------------------------------')
    t0 = datetime.now()

    train_tqdm_bar = tqdm(enumerate(train_loader), total = (len(train_loader)), leave = False, position = 0, file = sys.stdout, dynamic_ncols = True)
    val_tqdm_bar = tqdm(enumerate(validation_loader), total = (len(validation_loader)),  leave = False, position = 0, file = sys.stdout, dynamic_ncols = True)

    running_loss = 0.0
    running_corrects = 0.0
    val_running_loss = 0.0
    val_running_corrects = 0.0

    model.train()
    for idx, X in train_tqdm_bar:
        labels = X[2].to(device)
        iout, out = model(X)
        loss = criterion(out, labels)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1)
        optimizer.step()
        running_loss += loss
        running_corrects += torch.sum(iout == labels)
        train_tqdm_bar.set_description(desc = 'train'.format(epoch + 1, num_epochs))
        batch_idx = (idx + 1) * 64

        train_tqdm_bar.set_postfix(
            loss = running_loss.item() / batch_idx if idx + 1 < len(train_loader) else running_loss.item() / len(train_loader.dataset)
            ,acc = running_corrects.item() / batch_idx if idx + 1 < len(train_loader) else running_corrects.item() / len(train_loader.dataset)
            )
    
    model.eval()
    with torch.no_grad():
        for val_idx, v_X in val_tqdm_bar:
            v_labels = v_X[2].to(device)
            v_iout, v_out = model(v_X)
            v_loss = criterion(v_out, v_labels)
            val_running_loss += v_loss
            val_running_corrects += torch.sum(v_iout == v_labels)
            val_tqdm_bar.set_description('validate'.format(epoch + 1, num_epochs))
            val_batch_idx = (val_idx + 1) * 128

            val_tqdm_bar.set_postfix(
                val_loss = val_running_loss.item() / val_batch_idx if val_idx + 1 < len(validation_loader) else val_running_loss.item() / len(validation_loader.dataset)
                ,val_acc = val_running_corrects.item() / val_batch_idx if val_idx + 1 < len(validation_loader) else val_running_corrects.item() / len(validation_loader.dataset)
                )
    
    epoch_loss = running_loss/len(train_loader.dataset)
    losses.append(epoch_loss)
    epoch_accuracy = running_corrects/len(train_loader.dataset)
    accuracies.append(epoch_accuracy)
    val_epoch_loss = val_running_loss/len(validation_loader.dataset)
    val_losses.append(val_epoch_loss)
    val_epoch_accuracy = val_running_corrects/len(validation_loader.dataset)
    val_accuracies.append(val_epoch_accuracy)

    checkpoint = {
            'epoch': epoch + 1
            ,'state_dict': model.state_dict()
            ,'optimizer' : optimizer.state_dict()
            ,'val_loss_min' : val_epoch_loss
            }
    
    print('training loss: {:.4f}, acc: {:.2f}'.format(epoch_loss, epoch_accuracy))
    print('validation loss: {:.4f}, acc: {:.2f}'.format(val_epoch_loss, val_epoch_accuracy))
    print('elapsed time: {}'.format(str(datetime.now() - t0).split('.')[0]))

    if val_epoch_loss <= val_loss_min:
      print('validation loss decreased from {:.4f} to {:.4f}, saving model...'.format(val_loss_min, val_epoch_loss))
      # torch.save(checkpoint, 'checkpoint/ex2/question_pairs_lowest_val_loss_epoch_{}.pth'.format(epoch + 1))
      val_loss_min = val_epoch_loss

In [ ]:
plt.style.use('seaborn-darkgrid')
plt.figure()
plt.plot(accuracies, color = 'magenta')
plt.plot(val_accuracies, color = '#606060')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch #')
plt.legend(['train_acc', 'val_acc'], loc = 'lower right')
plt.grid(axis = 'y', c = 'black', alpha = 0.2)
plt.grid(axis = 'x', c = 'black', alpha = 0.2)

In [ ]:
plt.figure()
plt.plot(losses, color = 'magenta')
plt.plot(val_losses, color = '#606060')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch #')
plt.legend(['train_loss', 'val_loss'], loc = 'upper right')
plt.grid(axis = 'y', c = 'black', alpha = 0.2)
plt.grid(axis = 'x', c = 'black', alpha = 0.2)